In [1]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import torch
import timm
import csv
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import os
import itertools
from print_color import print
import time

In [2]:
def mip(image: np.ndarray)->np.ndarray:
    """Create the maximum intensity projection.
    """
    maximum_intensity_projection = np.max(image, axis=0)
    maximum_intensity_projection = np.clip(maximum_intensity_projection, -100, 300)
    
    return maximum_intensity_projection

In [3]:
def aip(image: np.ndarray)->np.ndarray:
    """Create a mean image projection.
    """
    average_intensity_projection = np.mean(image,axis=0)
    average_intensity_projection = np.clip(average_intensity_projection,-100,300)
    
    return average_intensity_projection

In [4]:
def find_tumor_center(mask: np.ndarray)->int:
    """Find the center of the tumor. 
    """
    return np.clip(np.argmax(np.sum(mask, axis=(1, 2))),-100,300)

In [5]:
def slice_tc(image: np.ndarray, mask: np.ndarray)->np.ndarray:
    """Take a slice at the center of the tumor.
    """
    x1 = find_tumor_center(mask)
    return np.clip(image[x1, :, :],-100,300)

In [6]:
def tumor_mip(image: np.ndarray, mask: np.ndarray)->np.ndarray:
    """Return the mip of the tumor only.
    """
    return np.clip(np.max(image * mask, axis=0),-100,300)

In [7]:
def minip(image: np.ndarray)->np.ndarray:
    """Create the maximum intensity projection.
    """
    return np.clip(np.min(image, axis=0),-100,300)


In [8]:
model = timm.create_model('convnext_large', pretrained=True)
model.eval()
model.to('cuda')

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (norm): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=192, out_features=768, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=768, out_features=192, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (norm): LayerNorm((192,), eps=1e-06, elemen

# 2D


In [74]:
start_time = time.time()

image = sitk.ReadImage('c:/users/sven/Documents/BEP/simplified_data\B04\IMAGES\B04.nii.gz')
img_data = sitk.GetArrayFromImage(image)

mask = sitk.ReadImage('c:/users/sven/Documents/BEP/simplified_data\B04\MASKS\B04.nii.gz')
mask_data = sitk.GetArrayFromImage(mask)

img_data = slice_tc(img_data, mask_data)
input_array = np.dstack([img_data,img_data,img_data])

# Convert numpy array to torch tensor and normalize
input_tensor = torch.from_numpy(np.transpose(input_array, (2, 0, 1))).float()
input_tensor = input_tensor.to('cuda')
input_tensor = (input_tensor + 100) / (300 + 100)
input_tensor = input_tensor.unsqueeze(0)

# Extract features using ResNet50
with torch.no_grad():
    features = model.forward_features(input_tensor)
    features = torch.mean(features, dim=[2, 3])

# Convert features to numpy array
features = features.to('cpu')
features_array = features.numpy()


end_time = time.time()
# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 0.45345473289489746 seconds


# 2D+


In [84]:
start_time = time.time()

image = sitk.ReadImage('c:/users/sven/Documents/BEP/simplified_data\B04\IMAGES\B04.nii.gz')
img_data = sitk.GetArrayFromImage(image)

mask = sitk.ReadImage('c:/users/sven/Documents/BEP/simplified_data\B04\MASKS\B04.nii.gz')
mask_data = sitk.GetArrayFromImage(mask)

img_data = tumor_mip(img_data, mask_data)
input_array = np.dstack([img_data,img_data,img_data])

# Convert numpy array to torch tensor and normalize
input_tensor = torch.from_numpy(np.transpose(input_array, (2, 0, 1))).float()
input_tensor = input_tensor.to('cuda')
input_tensor = (input_tensor + 100) / (300 + 100)
input_tensor = input_tensor.unsqueeze(0)

# Extract features using ResNet50
with torch.no_grad():
    features = model.forward_features(input_tensor)
    features = torch.mean(features, dim=[2, 3])

# Convert features to numpy array
features = features.to('cpu')
features_array = features.numpy()


end_time = time.time()
# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 0.5004169940948486 seconds


# 2D+ VI


In [94]:
start_time = time.time()

image = sitk.ReadImage('c:/users/sven/Documents/BEP/simplified_data\B04\IMAGES\B04.nii.gz')
img_data = sitk.GetArrayFromImage(image)

mask = sitk.ReadImage('c:/users/sven/Documents/BEP/simplified_data\B04\MASKS\B04.nii.gz')
mask_data = sitk.GetArrayFromImage(mask)

x1 = tumor_mip(img_data, mask_data)
x2 = mip(img_data)
x3 = aip(img_data)
input_array = np.dstack([x1,x2,x3])

# Convert numpy array to torch tensor and normalize
input_tensor = torch.from_numpy(np.transpose(input_array, (2, 0, 1))).float()
input_tensor = input_tensor.to('cuda')
input_tensor = (input_tensor + 100) / (300 + 100)
input_tensor = input_tensor.unsqueeze(0)

# Extract features using ResNet50
with torch.no_grad():
    features = model.forward_features(input_tensor)
    features = torch.mean(features, dim=[2, 3])

# Convert features to numpy array
features = features.to('cpu')
features_array = features.numpy()


end_time = time.time()
# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 0.5241715908050537 seconds
